# Modelling Student Success 

This project aims to create three models for predicting student success and measure them based on their accuracy.
The three models are below:

1. The Past Model:
Assesses how likely a pupils past and their parents successes are to contribute to a pupils grade.

Takes into account:
*Parents education,
Parents job,
Past failures of student,
Whether or not they attended nursery school,
Parents cohabitation status.*

2. The personal model:
Assess how likely pupil characteristics and social life/circumstances are to contribute to a pupils grade.

Takes into account:
*Sex,
Rural or urban living space,
Reason for choosing school,
Romantic relationship status,
Health,
Quality of family relationships,
Frequency of outings with friends.*

3. The academic model:
Assess how important academic life and exam performance rating is to pupil grades.

Takes into account:
*Whether a pupil had extra paid classes,
If a pupil went to extra curricular activities,
Period 1 grade,
Period 2 grade,
Absences,
Weekly study time,
Wants to take higher education.*

In [ ]:
import numpy as np
import pandas as pd 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import numpy as np
from matplotlib import pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Data comes in as two csv files so I chose to combine the math and portuegese students into one data frame.

In [ ]:
df_math = pd.read_csv("/kaggle/input/student-alcohol-consumption/student-mat.csv")
df_por = pd.read_csv("/kaggle/input/student-alcohol-consumption/student-por.csv")

# set math and por to binary
df_math["subject"] = "0"
df_por["subject"] = "1"

df = pd.concat([df_por, df_math])

The data is then split into three seperate data frames, one for each model.

In [ ]:
df_mod1 = df.loc[:, ['Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'failures', 'nursery', 'G3']]
df_mod2 = df.loc[:, ['sex', 'address', 'reason', 'romantic', 'health', 'famrel', 'goout', 'G3']]
df_mod3 = df.loc[:, ['G1', 'G2', 'paid', 'activities', 'absences', 'higher', 'studytime', 'G3']]

# Standardising Variables

I chose to set all text values to either binary -1 or 1 if there were two output options or to equal distances between -1 and 1 for wider categorical variables. The numerical variables are all standardised.

## **Model One**

In [ ]:
# Together T = 1 Apart A = 0
df_mod1['parent_status'] = df_mod1.apply(lambda row: -1 if row['Pstatus'] == "T" else 1, axis=1)
df_mod1 = df_mod1.drop('Pstatus', axis=1)

# set categorical variables to ranges -1 to 1
df_mod1.loc[df_mod1['Mjob'] == 'teacher', 'm_job'] = -1
df_mod1.loc[df_mod1['Mjob'] == 'health', 'm_job'] = -0.5
df_mod1.loc[df_mod1['Mjob'] == 'services', 'm_job'] = 0
df_mod1.loc[df_mod1['Mjob'] == 'at_home', 'm_job'] = 0.5
df_mod1.loc[df_mod1['Mjob'] == 'other', 'm_job'] = 1
df_mod1 = df_mod1.drop('Mjob', axis=1)

df_mod1.loc[df_mod1['Fjob'] == 'teacher', 'f_job'] = -1
df_mod1.loc[df_mod1['Fjob'] == 'health', 'f_job'] = -0.5
df_mod1.loc[df_mod1['Fjob'] == 'services', 'f_job'] = 0
df_mod1.loc[df_mod1['Fjob'] == 'at_home', 'f_job'] = 0.5
df_mod1.loc[df_mod1['Fjob'] == 'other', 'f_job'] = 1
df_mod1 = df_mod1.drop('Fjob', axis=1)

df_mod1.loc[df_mod1['Fedu'] == 0, 'f_edu'] = -1
df_mod1.loc[df_mod1['Fedu'] == 1, 'f_edu'] = -0.5
df_mod1.loc[df_mod1['Fedu'] == 2, 'f_edu'] = 0
df_mod1.loc[df_mod1['Fedu'] == 3, 'f_edu'] = 0.5
df_mod1.loc[df_mod1['Fedu'] == 4, 'f_edu'] = 1
df_mod1 = df_mod1.drop('Fedu', axis=1)

df_mod1.loc[df_mod1['Medu'] == 0, 'm_edu'] = -1
df_mod1.loc[df_mod1['Medu'] == 1, 'm_edu'] = -0.5
df_mod1.loc[df_mod1['Medu'] == 2, 'm_edu'] = 0
df_mod1.loc[df_mod1['Medu'] == 3, 'm_edu'] = 0.5
df_mod1.loc[df_mod1['Medu'] == 4, 'm_edu'] = 1
df_mod1 = df_mod1.drop('Medu', axis=1)

df_mod1.loc[df_mod1['failures'] == 0, 'times_failed'] = -1
df_mod1.loc[df_mod1['failures'] == 1, 'times_failed'] = -0.5
df_mod1.loc[df_mod1['failures'] == 2, 'times_failed'] = 0
df_mod1.loc[df_mod1['failures'] == 3, 'times_failed'] = 0.5
df_mod1.loc[df_mod1['failures'] == 4, 'times_failed'] = 1
df_mod1 = df_mod1.drop('failures', axis=1)

# Attended nursery 1 = Yes -1 = No
df_mod1['att_nursery'] = df.apply(lambda row: -1 if row['nursery'] == "no" else 1, axis=1)
df_mod1 = df_mod1.drop('nursery', axis=1)

# standard scale numeric values
col_names1 = ['G3']
features1 = df_mod1[col_names1]
scaler1 = StandardScaler().fit(features1.values)
features1 = scaler1.transform(features1.values)

df_mod1[col_names1] = features1


## **Model Two**

In [ ]:

# gender = M -1, F 1
df_mod2['gender'] = df_mod2.apply(lambda row: -1 if row['sex'] == "M" else 1, axis=1)
df_mod2 = df_mod2.drop('sex', axis=1)

# rural_urban = Urban -1, Rural 1
df_mod2['rural_urban'] = df_mod2.apply(lambda row: -1 if row['address'] == "U" else 1, axis=1)
df_mod2 = df_mod2.drop('address', axis=1)

# set categorical variables to be between -1 and 1
df_mod2.loc[df_mod2['reason'] == 'home', 'choice'] = -1
df_mod2.loc[df_mod2['reason'] == 'reputation', 'choice'] = -0.5
df_mod2.loc[df_mod2['reason'] == 'course', 'choice'] = 0.5
df_mod2.loc[df_mod2['reason'] == 'other', 'choice'] = 1
df_mod2 = df_mod2.drop('reason', axis=1)

df_mod2.loc[df_mod2['health'] == 1, 'healthy'] = -1
df_mod2.loc[df_mod2['health'] == 2, 'healthy'] = -0.5
df_mod2.loc[df_mod2['health'] == 3, 'healthy'] = 0
df_mod2.loc[df_mod2['health'] == 4, 'healthy'] = 0.5
df_mod2.loc[df_mod2['health'] == 5, 'healthy'] = 1
df_mod2 = df_mod2.drop('health', axis=1)

df_mod2.loc[df_mod2['famrel'] == 1, 'family_rel'] = -1
df_mod2.loc[df_mod2['famrel'] == 2, 'family_rel'] = -0.5
df_mod2.loc[df_mod2['famrel'] == 3, 'family_rel'] = 0
df_mod2.loc[df_mod2['famrel'] == 4, 'family_rel'] = 0.5
df_mod2.loc[df_mod2['famrel'] == 5, 'family_rel'] = 1
df_mod2 = df_mod2.drop('famrel', axis=1)

df_mod2.loc[df_mod2['goout'] == 1, 'out_freq'] = -1
df_mod2.loc[df_mod2['goout'] == 2, 'out_freq'] = -0.5
df_mod2.loc[df_mod2['goout'] == 3, 'out_freq'] = 0
df_mod2.loc[df_mod2['goout'] == 4, 'out_freq'] = 0.5
df_mod2.loc[df_mod2['goout'] == 5, 'out_freq'] = 1
df_mod2 = df_mod2.drop('goout', axis=1)

# relationship = Yes 1, No -1
df_mod2['relationship'] = df_mod2.apply(lambda row: 1 if row['romantic'] == "yes" else -1, axis=1)
df_mod2 = df_mod2.drop('romantic', axis=1)

# standard scale numeric values
col_names2 = ['G3']
features2 = df_mod2[col_names2]
scaler2 = StandardScaler().fit(features2.values)
features2 = scaler2.transform(features2.values)

df_mod2[col_names2] = features2


## **Model Three**

In [ ]:
# paid_classes = 1 Yes, -1 No
df_mod3['paid_classes'] = df_mod3.apply(lambda row: 1 if row['paid'] == "yes" else -1, axis=1)
df_mod3 = df_mod3.drop('paid', axis=1)

# att_extra = 1 Yes -1 No
df_mod3['att_extra'] = df_mod3.apply(lambda row: 1 if row['activities'] == "yes" else -1, axis=1)
df_mod3 = df_mod3.drop('activities', axis=1)

# higher_ed = 1 Yes -1 No
df_mod3['higher_ed'] = df_mod3.apply(lambda row: 1 if row['higher'] == "yes" else -1, axis=1)
df_mod3 = df_mod3.drop('higher', axis=1)

# standard scale numeric values
col_names3 = ['G3', 'G1', 'G2', 'absences']
features3 = df_mod3[col_names3]
scaler3 = StandardScaler().fit(features3.values)
features3 = scaler3.transform(features3.values)

df_mod3[col_names3] = features3

# **Training the Model**

I chose to use linear regression to train the model with the sklearn package.

## Splitting Train and Test Data

In [ ]:
y_1 = df_mod1.G3
y_2 = df_mod2.G3
y_3 = df_mod3.G3

df_mod1 = df_mod1.drop('G3', axis=1)
df_mod2 = df_mod2.drop('G3', axis=1)
df_mod3 = df_mod3.drop('G3', axis=1)

x_train_mod_1, x_test_mod_1, y_train_mod_1, y_test_mod_1 = train_test_split(df_mod1, y_1, test_size=0.2)
x_train_mod_2, x_test_mod_2, y_train_mod_2, y_test_mod_2 = train_test_split(df_mod2, y_2, test_size=0.2)
x_train_mod_3, x_test_mod_3, y_train_mod_3, y_test_mod_3 = train_test_split(df_mod3, y_3, test_size=0.2)

## Fitting model and display coefficients

In [ ]:
lm = LinearRegression()
model_1 = lm.fit(x_train_mod_1, y_train_mod_1)
coefficients1 = pd.concat([pd.DataFrame(df_mod1.columns),pd.DataFrame(np.transpose(lm.coef_))], axis = 1)
print(coefficients1)
predictions1 = lm.predict(x_test_mod_1)

model_2 = lm.fit(x_train_mod_2, y_train_mod_2)
coefficients2 = pd.concat([pd.DataFrame(df_mod2.columns),pd.DataFrame(np.transpose(lm.coef_))], axis = 1)
print(coefficients2)
predictions2 = lm.predict(x_test_mod_2)

model_3 = lm.fit(x_train_mod_3, y_train_mod_3)
coefficients3 = pd.concat([pd.DataFrame(df_mod3.columns),pd.DataFrame(np.transpose(lm.coef_))], axis = 1)
print(coefficients3)
predictions3 = lm.predict(x_test_mod_3)

We can see from the model coefficients the models each have different variables that have stronger effects. 

Model one seems to be most affected by the number of times a student failed which makes the most sense as it is the most direct measure as opposed to more abstract things like whether parents are still together.

Model two seems to have the weakest coefficients though the strongest of these are where the student lives and how often they go out; with students in urban areas who go out less having better results this makes sense as they have a wider access to resources and more time to study.

Model three however has strong coefficients especially for G2 this is expected as it represents a pupils grade in period two so is a direct measure of the same thing we are trying to predict (G3) just a few months earlier. It appears though that the attendance of extra classes wasn't too helpful this could indicate the pupils in these extra classes were struggling anyway.

# Analysing and interpreting results

## Model One

In [ ]:
# print and plot results
print('Mean Absolute Error 1:', metrics.mean_absolute_error(y_test_mod_1, predictions1))
print('Mean Squared Error 1:', metrics.mean_squared_error(y_test_mod_1, predictions1))
print('Root Mean Squared Error 1:', np.sqrt(metrics.mean_squared_error(y_test_mod_1, predictions1)))


plt.scatter(y_test_mod_1, predictions1)
plt.ylim(bottom=-3, top=2)
plt.ylabel('Test Data')
plt.xlabel('Predictions')
plt.show()

Model one has a high error and the plot shows that the predictions were mostly off. The weak correlation between the students historic circumstances and their final grade leads me to discredit this model as inaccurate. There is no clear relationship between the predictions and the test data and to highlight this the RMSE too is very high.

## Model Two

In [ ]:
print('Mean Absolute Error 2:', metrics.mean_absolute_error(y_test_mod_2, predictions2))
print('Mean Squared Error 2:', metrics.mean_squared_error(y_test_mod_2, predictions2))
print('Root Mean Squared Error 2:', np.sqrt(metrics.mean_squared_error(y_test_mod_2, predictions2)))

plt.scatter(y_test_mod_2, predictions2)
plt.ylim(bottom=-3, top=2)
plt.ylabel('Test Data')
plt.xlabel('Predictions')
plt.show()

This model was formed using the pupils personal circumstances like relationship status and living area. Again the predictions are weak the RMSE shows high variance between the predictions and test values which allows the conclusion to be drawn that the personal circumstances measured across this sample of pupils had little affect on their results.

## Model Three

In [ ]:
print('Mean Absolute Error 3:', metrics.mean_absolute_error(y_test_mod_3, predictions3))
print('Mean Squared Error 3:', metrics.mean_squared_error(y_test_mod_3, predictions3))
print('Root Mean Squared Error 3:', np.sqrt(metrics.mean_squared_error(y_test_mod_3, predictions3)))

plt.scatter(y_test_mod_3, predictions3)
plt.ylim(bottom=-3, top=2)
plt.ylabel('Test Data')
plt.xlabel('Predictions')
plt.show()

This model has the lowest error output. The graph also shows some correlation between the predictions and test values. The modelling data used was the past academic performance of the pupil as well as other activities undertaken in their academic life. We saw above that the G2 (grade in period two) was the most heavily relied upon coefficient and combined with the others we get a failry accurate picture of how most pupils will perform. However the model is not perfect for example there are a few pupils who are predicted to perform very poorly who ended up being fairly average that would require further examination. 

# Conclusion

The aim of the project was to determine what values work best to determine a prediction for a pupils final grade. The first lesson learned was that a pupils personal and family history have very little use when trying to determine how far a pupil will go meaning the son of a married teacher is just as likely to over-achieve or indeed under-achieve as the daughter of a divorced stay at home parent.

The second lesson learned was when predicting pupil results as with anything it's best (as with anything else) to work from the problem not from the photograph and use the closest data that accurately models the same system. In this case the best feature selected was grade during term two (G2) which is the most direct comparison as opposed to quality of family relationships (famrel) which is fairly abstract.

Finally even though we decided model three (academic life) was the most accurate, it wasn't perfect. The students predicted to perform poorly mostly due to their G2 grade who then bootstrapped their learning or performed better on exam day in G3 (period three) were inaccurately placed by the regression. This to me seems like a feedback system in which the pupils found out how they performed in G2 and used the feedback to change the outcome.

The thing I would recommend coming out of the project is a message of acceptance, pupils from all backgrounds are equally capable or performance and under performance. The predictive power of any model is limited by the data it holds but so are human brains. The students who performed poorly and improved were able to use the feedback to their advantage and in future pupils who don't feel limited by social circumstance can out-perform every previous iteration of student that came before them.